* SVM基本概念(关键术语)
* SVM实现$\longrightarrow$序列最小优化(Sequential Minimal Optimization,SMO)算法
* SVM扩展到更多数据集的方式$\longrightarrow$核函数(kernel)
* 回顾C1中的手写识别例子，考察能否通过SVM提高识别效果

#### 6.1 基于最大间隔分隔数据

支持向量(support vector)就是离分隔超平面最近的那些点。接下来要试着最大化支持向量到分割面的距离，需要找到此问题的优化求解方法。

#### 6.2 寻找最大间隔

待整理~~

#### 6.3 SMO高效优化算法

SMO算法工作原理：每次循环中选择两个alpha进行优化处理。一旦找到一对合适的alpha，那么就增大其中一个同时减小另一个。

这里所谓的“合适”就是指两个alpha必须要符合一定的条件：
* 条件一就是这两个alpha必须要在间隔边界之外。
* 条件二则是这两个alpha还没有进行过区间化处理或者不在边界上。

In [1]:
import numpy as np

def loadDataSet(fileName):
    dataMat=[]
    labelMat=[]
    fr=open(fileName)
    for line in fr.readlines():
        lineArr=line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

#返回一个与输入值i不一样的随机生成的alpha下表j（m是所有alpha的数目）
def selectJrand(i,m):
    j=i
    while (j==i):
        j=int(np.random.uniform(0,m))
    return j

#用于调整大于H或小于L的alpha值，即aj取值范围(L,H)
def clipAlpha(aj,H,L):
    if aj>H:
        aj=H
    if L>aj:
        aj=L
    return aj

In [3]:
dataArr,labelArr=loadDataSet('Ch05/testSet.txt')
labelArr     #类别标签是-1、1

[0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0]

In [36]:
#简化版SMO算法
def smoSimple(dataMatIn,classLabels,C,toler,maxIter):
    dataMatrix=np.mat(dataMatIn)                  #转换为numpy矩阵，可以简化数学处理操作
    labelMat=np.mat(classLabels).transpose()      #转置类标签，得到列向量，即与数据矩阵中的每行样本一一对应
    b=0
    m,n=np.shape(dataMatrix)                      #得到数据矩阵的shape属性
    alphas=np.mat(np.zeros((m,1)))                #构建一个alpha列矩阵，元素初始化0
    iter=0                                        #存储在没有任何alpha改变的情况下遍历数据集的次数。当该变量达到maxIter时，函数结束运行
    while (iter<maxIter):        #当alpha没有任何改变的情况下遍历数据集次数还不满足最大迭代次数时运行以下代码
        alphaPairsChanged=0      #用于记录alpha是否已经进行优化
        for i in range(m):
            fXi=float(np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T))+b   #np.multiply相应元素乘法
            Ei=fXi-float(labelMat[i])
            if ((labelMat[i]*Ei < -toler) and (alphas[i]< C)) or ((labelMat[i]*Ei>toler) and (alphas[i]>0)):
                j=selectJrand(i,m)
                fXj=float(np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T))+b
                Ej=fXj-float(labelMat[j])
                alphaIold=alphas[i].copy()
                alphaJold=alphas[j].copy()
                if (labelMat[i]!=labelMat[j]):
                    L=max(0,alphas[j]-alphas[i])
                    H=min(C,C+alphas[j]-alphas[i])
                else:
                    L=max(0,alphas[j]+alphas[i]-C)
                    H=min(C,alphas[j]+alphas[i])
                
                if L==H:
                    print('L==H')
                    continue   #跳出当前循环的剩余代码
                
                eta=2*dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T
                if eta>=0:
                    print('eta>=0')
                    continue
                
                alphas[j] -= labelMat[j]*(Ei-Ej)/eta
                alphas[j]=clipAlpha(alphas[j],H,L)
                if (abs(alphas[j]-alphaJold)<0.00001):
                    print('j not moving enough')
                    continue
                
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold-alphas[j])
                b1=b-Ei-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T-labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2=b-Ej-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T-labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0<alphas[i]) and (C>alphas[i]):
                    b=b1
                elif (0<alphas[j]) and (C>alphas[i]):
                    b=b2
                else:
                    b=(b1+b2)/2
                alphaPairsChanged+=1
                print('iter: %d i:%d,pairs changed %d'%(iter,i,alphaPairsChanged))
            if (alphaPairsChanged==0):
                iter+=1
            else:
                iter=0
            print('iteration number: %d'% iter)
        return b,alphas

In [37]:
b,alphas=smoSimple(dataArr,labelArr,0.6,0.001,40)

iteration number: 1
L==H
iteration number: 2
iteration number: 3
iteration number: 4
j not moving enough
iteration number: 5
L==H
iteration number: 6
iteration number: 7
L==H
iteration number: 8
L==H
iteration number: 9
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
L==H
L==H
L==H
iteration number: 10
j not moving enough
L==H
iteration number: 11
iteration number: 12
L==H
L==H
iteration number: 13
L==H
j not moving enough
iteration number: 14
j not moving enough
j not moving enough
iteration number: 15
iteration number: 16
iteration number: 17
iteration number: 18
iteration number: 19
L==H
j not moving enough
iteration number: 20
L==H
L==H
iteration number: 21
j not moving enough
j not moving enough
iteration number: 22
iteration number: 23
iteration number: 24
iteration number: 25
iteration number: 26
iteration number: 27
j not moving enough
L==H
iteration number: 28
j not moving enough
iteration number: 29
L==H
j not moving enough
L==H
iteration 